# Homework 1

### 1. Concept Questions
Code below is for 1.5

In [ ]:
import numpy as np

A = np.array([[0,1,1,0,0], [1,0,1,0,0], [1,1,0,0,0], [0,0,0,0,1], [0,0,0,1,0]])
D = np.array([[2,0,0,0,0], [0,2,0,0,0], [0,0,2,0,0], [0,0,0,1,0], [0,0,0,0,1]])
L = D-A

# Eigenvalue decomposition
s, v = np.linalg.eig(L)

# Get index of eigenvalues that are 0
zero_evalue_index =  [i for i, x in enumerate(np.around(s,1)) if x == 0]
print("Eigenvalues: " + str(np.around(s,1)))
print("Indexes of 0 eigenvalues: " + str(zero_evalue_index))

# Columns corresponding to the eigenvalues that are 0
print("Eigenvectors: ")
print(str(v))
print("Cluster Assignment:")
print(str(v[:, zero_evalue_index]))

Based on the resulting matrix, I can conclude that nodes 1, 2 and 3 are connected in one cluster while nodes 4 and 5 are connected in another. 

## 2. Image Compression Using Clustering

### Q1

In [ ]:
import imageio  
import matplotlib.image
import random 
import numpy as np
import time
from datetime import timedelta
from collections import defaultdict

In [ ]:
###### Define Functions ######
def assign_cluster(pxl, c, norm=2):
    """
    Calculates distance between single pixel and list of centroids.

    Input:
        pxl = pixel [R,G,B]
        c = list of centroids
        norm = L1 or L2, default is L2
    Output:
        Cluster Index
    
    Author's Note: 
    This function is what takes the longest when I run the code. 
    I couldn't figure out how to optimize, but it still works! 
    
    """    
    min_distance = 0
    cluster_index = 0
    
    # Loop through list of centroids
    for i in range(len(c)):
        x = pxl-c[i]

        if norm==1:
            distance = np.linalg.norm(x, 1)
        else:
            distance = np.linalg.norm(x, 2)
            
        if i == 0:
            min_distance = distance
            cluster_index = i
        else:
            if distance < min_distance:
                cluster_index = i
                min_distance = distance            
    
    return cluster_index

def adjust_centroids(pixel, cluster_assign, centroids, norm=2):
    """
    Adjusts the centroids based on mean (L2 norm) or median (L1 norm)
    
    Input
        pixel = RGB representation of image
        cluster_assign = cluster assignment
        centroids = current list of centroids
    Output
        adjusted centroid list 
    """
        
    m = pixel.shape[0]
    n = pixel.shape[1]
    d = pixel.shape[2]
    
    new_centroids = defaultdict()
    centroid_list = []
    final_centroids = []
    
    # Loop through pixels in image. Add pixel to corresponding cluster list
    for i in range(m):
        for j in range(n):
            pxl = pixel[i,j]
            cluster = cluster_assign[i,j] 
            
            if cluster not in new_centroids:
                new_centroids[cluster] = list()
                
            new_centroids[cluster].append(pxl)  
            
    # Calculate the new list of centroids
    for cluster in new_centroids:
        centroid_list = np.array(new_centroids[cluster])
        
        if norm==1:
            updated_centroid = np.median(centroid_list, axis=0).astype(np.uint8)
        else:
            updated_centroid = np.around(centroid_list.sum(axis=0)/len(centroid_list)).astype(np.uint8)
        
        final_centroids.append(updated_centroid.tolist())
    
    return final_centroids 

def run_kmeans(pixel, k, norm=2):
    """
    Runs the kmeans algorithm.
    
    Input:
        pixel = RGB representation of image
        k = number of desired clusters
        norm = distance measure, can be 1 or 2
    
    Output:
        cluster labels
        new centroids 
        compressed image 
        number of iterations 
        run time
    """    
    start_time = time.monotonic()
    
    m = pixel.shape[0] 
    n = pixel.shape[1] 
    d = pixel.shape[2] 
    cluster_assign = np.empty(shape=(m,n),dtype='object')
    
    np.random.seed(206)
    
    # Initialize random centroids
    c_r = np.random.randint(0, 255, size = k)
    c_g = np.random.randint(0, 255, size = k)
    c_b = np.random.randint(0, 255, size = k)
    centroids = np.array(list(zip(c_r, c_g, c_b))).tolist()
    centroids.sort()
    
    # Initialize the clusters
    cluster_assign = np.empty(shape=(m,n),dtype='object')

    iter_count = 1

    # Assign the cluster to each pixel for the first time
    for i in range(m):
        for j in range(n):
            pxl = pixel[i][j]
            cluster_assign[i,j] = assign_cluster(pxl, centroids, norm)
        
    # Get new centroids
    new_centroids = adjust_centroids(pixel, cluster_assign, centroids, norm) 
    new_centroids.sort()
    
    while centroids != new_centroids: # Keep looping until the new centroids are equal to the old centroids
        iter_count += 1
        
        centroids = new_centroids

        for i in range(m):
            for j in range(n):
                pxl = pixel[i][j]
                cluster_assign[i,j] = assign_cluster(pxl, centroids, norm)
                        
        # Get new centroids
        new_centroids = adjust_centroids(pixel, cluster_assign, centroids, norm) 
        new_centroids.sort()
        
    # Time Calculation
    end_time = time.monotonic()
    time_diff = timedelta(seconds=end_time - start_time).seconds
    
    # Create compressed image
    final_image = np.empty(shape=(m,n,d),dtype='object')
    for row in range(m):
        for col in range(n):
            pxl = cluster_assign[row][col] 
            final_image[row,col] = np.array(new_centroids[pxl])   
    
    # Final Outputs
    cluster_labels = cluster_assign+1
    final_image = np.reshape(final_image, (pixel.shape))
    
    return cluster_labels, new_centroids, final_image, iter_count, time_diff

In [ ]:
# Loop through pictures and k values using norm = 2

k_vals = [2,4,8,16]

image = ['data/GeorgiaTech.bmp', 'data/football.bmp', 'data/beach.jpg']

for k in k_vals:
    for pic in image:  
        
        # Read image
        original_image =  imageio.imread(pic)
        
        # Run kmeans
        cluster_label, cluster_center, comp_image, iterations, run_time = run_kmeans(original_image, k, norm=2)
        
        # Export file
        pic_name = pic.split('/')[1].split('.')[0]
        matplotlib.image.imsave('Q2_output/'+pic_name+'_'+str(k)+'.png', comp_image.astype(np.uint8))
        
        print(pic_name + ', k = ' + str(k) + ', Iterations = ' + str(iterations) + ', Run Time = ' + str(run_time) + ' secs.')

### Q2

In [ ]:
# Loop through pictures and k values using norm = 1
k_vals = [2,4,8,16]

image = ['data/GeorgiaTech.bmp', 'data/football.bmp', 'data/beach.jpg']

for k in k_vals:
    for pic in image:  
        
        # Read image
        original_image =  imageio.imread(pic)
        
        # Run kmeans
        cluster_label, cluster_center, comp_image, iterations, run_time = run_kmeans(original_image, k, norm=1)
        
        # Export file
        pic_name = pic.split('/')[1].split('.')[0]
        matplotlib.image.imsave('Q2_output/Part_2/'+pic_name+'_'+str(k)+'.png', comp_image.astype(np.uint8))
        
        print(pic_name + ', k = ' + str(k) + ', Iterations = ' + str(iterations) + ', Run Time = ' + str(run_time) + ' secs.')

In [ ]:
# Loop through pictures and k values using norm = 1

k_vals = [16]

image = ['data/beach.jpg', 'data/football.bmp' ]

for k in k_vals:
    for pic in image:  
        
        # Read image
        original_image =  imageio.imread(pic)
        
        # Run kmeans
        cluster_label, cluster_center, comp_image, iterations, run_time = run_kmeans(original_image, k, norm=1)
        
        # Export file
        pic_name = pic.split('/')[1].split('.')[0]
        matplotlib.image.imsave('Q2_output/Part_2/'+pic_name+'_'+str(k)+'.png', comp_image.astype(np.uint8))
        
        print(pic_name + ', k = ' + str(k) + ', Iterations = ' + str(iterations) + ', Run Time = ' + str(run_time) + ' secs.')

## Political Blogs Dataset

### Q1

In [ ]:
import csv
from sklearn.cluster import KMeans
from collections import defaultdict
from statistics import mode

##### Define Functions #####
def run_spec(L, k):
    """
    Performs eigenvalue/eigenvector decomposition and performs kmeans based on value of k.
    
    Input:
        L = Laplacian 
        k = number of clusters
    Output:
        kmeans object
        cluster labels   
        
    Author's Note:
    Code was borrowed from test_football.py
    
    """
    # Eigenvector Decomposition
    v, x = np.linalg.eig(L)
    idx = np.argsort(v)
    x = np.real(x[:, idx[-k:]]) # select the k largest eigenvectors
    x = x/np.repeat(np.sqrt(np.sum(x*x, axis=1).reshape(-1, 1)), k, axis=1) # ensure all vectors are of unit length
    
    # Run kmeans
    kmeans = KMeans(n_clusters=k).fit(x)
    c_idx = kmeans.labels_

    return kmeans, c_idx

def calc_metrics(nodes, labels):
    """
    Calculates majority labels and mismatch rates for clusters.
    
    Input:
        Nodes = node list
        Labels = cluster labels (output from run_spec)
    Output:
        majority = dictionary, {cluster: majority political orientation}
        mismatch = dictionary, {cluster: mismatch rate}
    """
    
    num_nodes = nodes.shape[0]
    cluster_groups = defaultdict()
    majority = {}
    mismatch = {}
    
    # Collect political orientation of blogs associated with each cluster
    for i in range(num_nodes):
        node = nodes[i]
        group = node[2].astype(int)
        cluster = labels[i]
        
        if cluster not in cluster_groups:
            cluster_groups[cluster] = list()
        
        cluster_groups[cluster].append(group)
            
    # Majority Label
    for key in cluster_groups.keys():
        majority[key]=mode(cluster_groups[key])
    
    # Calculate Match Rate
    for key in cluster_groups.keys():
        value = cluster_groups[key]
        most_common = majority[key]
        
        total = len(value)
        count = 0
        
        # Calculate Mismatch Rate
        for i in range(total):
            if value[i] == most_common:
                count += 1
                        
        # Return mismatch rate to dictionary
        mismatch[key] = float(format(1 - count/total, '.4f'))
        
    return majority, mismatch 

In [ ]:
# Load txt files
# Source: https://stackoverflow.com/questions/16989647/importing-large-tab-delimited-txt-file-into-python/16999000

with open('data/nodes.txt') as f:
    reader = csv.reader(f, delimiter="\t")
    nodes0 = list(reader)  # Schema = ["Node", "Site", "Orientation", "Tags"]

with open('data/edges.txt') as f:
    reader = csv.reader(f, delimiter="\t")
    edges0 = list(reader) # Schema = Start, End

# Change data types
edges = np.array(edges0).astype(int)

nodes = nodes0
for i in range(len(nodes0)):   
    nodes[i][0] = int(nodes0[i][0])
    nodes[i][2] = int(nodes0[i][2])

# Create Adjacency Matrix 
n = len(nodes)
A = np.zeros(shape=(n,n),dtype='object')

for edge in edges:
    i = (edge-1)[0]
    j = (edge-1)[1]
    A[i,j] = 1
    A[j,i] = 1

# Create Degree matrix
d_i = A.sum(axis=0)
D = np.diagflat(d_i)

# Get index of nodes that don't have any connections
del_list = []

for i in range(len(d_i)):
    if d_i[i] == 0:
        del_list.append(i)
        
# Remove corresponding rows and columns from D, A and nodes
A = np.delete(A, del_list, 0)
A = np.delete(A, del_list, 1)
D = np.delete(D, del_list, 0)
D = np.delete(D, del_list, 1)
nodes = np.delete(nodes, del_list, 0)

# Calculate Laplacian: L = D-A
L = (D-A).astype(float)


In [ ]:
# Loop through different values of k
k_vals = [2,5,10,20]

for k in k_vals:
    kmeans_obj, labels = run_spec(L, k)
    majority, mismatch = calc_metrics(nodes, labels)
    
    avg = mean(mismatch.values())
    min_rate = min(mismatch.values())
    max_rate = max(mismatch.values())
    
    print("k = " + str(k))
    print("Majority: " + str(dict(sorted(majority.items()))))
    print("Mismatch Rates: " + str(dict(sorted(mismatch.items()))))
    print("Average of Mismatch Rates = " + str(avg))
    print("Spread of Mismatch Rates = (" + str(min_rate) + ", " + str(max_rate) + ") = " + str(max_rate-min_rate))
    print("")

### Q2

In [ ]:
for k in range(2,10):
    kmeans_obj, labels = run_spec(L, k)
    majority, mismatch = calc_metrics(nodes, labels)
    
    avg = mean(mismatch.values())
    min_rate = min(mismatch.values())
    max_rate = max(mismatch.values())
    
    print("k = " + str(k))
    print("Majority: " + str(dict(sorted(majority.items()))))
    print("Mismatch Rates: " + str(dict(sorted(mismatch.items()))))
    print("Average of Mismatch Rates = " + str(avg))
    print("Spread of Mismatch Rates = (" + str(min_rate) + ", " + str(max_rate) + ") = " + str(max_rate-min_rate))
    print("")